# Constructing
In order to process input data easly, We may want to construct the 3D representation from input data. For example, we can create a point cloud from a depth image to apply point cloud processing such as filtering and ICP. This section introduce method to construct 3D and 2.5D representations from other representations. 

The section use methods as following:
- Triangle mesh to a point cloud
- Depth image to a point cloud
- Voxel to a point cloud


In [1]:
%load_ext autoreload
%autoreload 2

## Triangle mesh to a point cloud

This subsection intorduce a method to construct a point cloud from a triangle mesh model. The subsection tutorial use following code:

In [2]:
from tutlibs.constructing import mesh_to_point
from tutlibs.io import Mesh
from tutlibs.visualization import JupyterVisualizer as jv

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
vertices, triangles, data = Mesh.read("../data/bunny_tm.ply")
point_cloud = mesh_to_point(vertices, triangles, 2000)
obj_points = jv.point(point_cloud, point_size=0.005)
jv.display([obj_points])

Output()

This output show a point cloud sampled from a triangle mesh model.

## Depth image to a point cloud
This subsection intorduce a method to construct a point cloud from a depth image. The subsection tutorial use following code:

In [6]:
import cv2
from tutlibs.constructing import depth_to_point
from tutlibs.visualization import JupyterVisualizer as jv

In [7]:
fx = fy = 525
cx = 319.5
cy = 239.5

depth_image = cv2.imread("../data/redwood_3dscan_depth.png", -1)
point_cloud, _ = depth_to_point(depth_image, fx, fy, cx, cy)
obj_points = jv.point(point_cloud)
jv.display([obj_points])

Output()

This output show a point cloud sampled from a depth image.

## Voxel to a point cloud
This subsection intorduce a method to construct a point cloud from a voxel model. The subsection tutorial use following code:

In [8]:
import numpy as np
from tutlibs.constructing import voxel_to_point
from tutlibs.visualization import JupyterVisualizer as jv

In [11]:
voxels = np.load("../data/bunny_vox.npy")
point_cloud = voxel_to_point(voxels)
obj_points = jv.point(point_cloud, point_size=0.5)
jv.display([obj_points])

Output()

This output show a point cloud sampled from a voxel model.